# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [21]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [22]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,san luis de la loma,17.2706,-100.8939,28.50,47,3,3.64,MX,1711543295
1,1,trairi,-3.2778,-39.2689,24.27,94,100,2.01,BR,1711528771
2,2,ribeira grande,38.5167,-28.7000,15.22,63,75,17.49,PT,1711525605
3,3,nova sintra,14.8667,-24.7167,19.61,76,15,6.22,CV,1711525055
4,4,kapa'a,22.0752,-159.3190,25.99,80,60,2.24,US,1711557236


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [23]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points("Lng", "Lat", geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City")

# Display the map

map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [24]:
# Narrow down cities that fit criteria and drop any results with null values
temp_df = city_data_df.loc[(city_data_df['Max Temp'] > 20)  & (city_data_df['Max Temp'] < 27)]
wind_df = temp_df.loc[temp_df['Wind Speed'] < 4]
vacation_df = wind_df.loc[wind_df['Cloudiness'] < 5]

# Drop any rows with null values
vacation_df = vacation_df.dropna(how = "any")

# Display sample data
vacation_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
30,30,hinthada,17.6333,95.4667,24.33,56,0,2.23,MM,1711582541
99,99,ingeniero jacobacci,-41.3427,-69.5448,21.80,24,0,3.70,AR,1711536532
185,185,taoudenni,22.6783,-3.9836,26.18,12,0,2.58,ML,1711519971
273,273,gomoh,23.8667,86.1667,26.60,20,2,1.87,IN,1711584671
283,283,guerrero negro,27.9769,-114.0611,20.40,45,0,2.24,MX,1711546283
299,299,atlanta,33.7490,-84.3880,23.09,49,0,1.34,US,1711539059
371,371,tabas,33.5959,56.9244,20.22,23,4,0.85,IR,1711505179
410,410,livingstone,-17.8419,25.8543,23.15,50,4,2.29,ZM,1711513308
438,438,tindouf,27.6711,-8.1474,24.25,12,0,2.57,DZ,1711520889
460,460,rakops,-21.0167,24.3333,26.29,39,0,2.73,BW,1711513710


### Step 3: Create a new DataFrame called `hotel_df`.

In [25]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
vacation_copy = vacation_df.copy()
hotel_df = vacation_copy[['City','Country','Lat','Lng','Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df.loc[:, "Hotel Name"] = ""
# Display sample data
hotel_df

/var/folders/zg/4brng8w568j1m9x6tsv0dhrc0000gn/T/ipykernel_66558/3170244649.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df.loc[:, "Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
30,hinthada,MM,17.6333,95.4667,56,
99,ingeniero jacobacci,AR,-41.3427,-69.5448,24,
185,taoudenni,ML,22.6783,-3.9836,12,
273,gomoh,IN,23.8667,86.1667,20,
283,guerrero negro,MX,27.9769,-114.0611,45,
299,atlanta,US,33.7490,-84.3880,49,
371,tabas,IR,33.5959,56.9244,23,
410,livingstone,ZM,-17.8419,25.8543,50,
438,tindouf,DZ,27.6711,-8.1474,12,
460,rakops,BW,-21.0167,24.3333,39,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [26]:
# Set parameters to search for a hotel
radius = 10,000
params = {
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = # YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

SyntaxError: invalid syntax (788381735.py, line 2)

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)